# Tratamento dos dados 

## Bibliotecas

In [1]:
import pandas as pd 
import numpy as np 
import scipy 
import _pickle as cPickle
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition

## Importando os dados

In [2]:
df = pd.read_csv('pov19ce.csv')

## Limpando os dados

### Excluindo colunas

In [3]:
df = df.drop(['Unnamed: 0', 'hh_id', 'pobre_ex'], axis = 1)

### Eliminar os missings values

In [4]:
df = df.dropna(subset=['esgoto_ad'])

### Definindo listas de nomes

#### Definição de uma lista com nomes das variáveis categoricas

In [5]:
cat_vars = ['area', 'casal', 'genero', 'raca', 'escolaridade', 'atividade', 'aposentadoria', 
            'dom_tipo', 'agua_adeq', 'esgoto_ad', 'banheiro', 'paredes', 'casa_tipo', 
            'geladeira', 'maquina_lavar', 'tv', 'computador', 'internet', 'carro', 'moto']

#### Definição de uma lista com nomes das variáveis numéricas


In [6]:
num_vars = ['idade', 'n_moradores', 'n_criancas', 'n_adultos', 'densidade', 'fone_pc']

### Filtrando colunas 


In [7]:
df = df[['pobre', 'area', 'casal', 'genero', 'raca', 'escolaridade', 'atividade', 'aposentadoria', 
                   'dom_tipo', 'agua_adeq', 'esgoto_ad', 'banheiro', 'paredes', 'casa_tipo', 
                   'geladeira', 'maquina_lavar', 'tv', 'computador', 'internet', 'carro', 'moto', 
                   'idade', 'n_moradores', 'n_criancas', 'n_adultos', 'densidade', 'fone_pc']]

### Criando dummy para a variavel e resposta

In [8]:
df['pobre'] = pd.get_dummies(df.pobre, drop_first=True)


## Exportando os dados Limpos 

In [9]:
df.to_csv("pov19ce_clean.csv")

## Pre Tratamento 

### Divisão da amostra 

In [10]:
df_train, df_test = model_selection.train_test_split(df, test_size=0.25, stratify= df.pobre, random_state=1)

### Ajustar os dados de treino

In [11]:
X_train = df_train.drop('pobre', axis=1)
y_train = df_train['pobre']
X_test = df_test.drop('pobre',axis=1)
y_test = df_test['pobre']

### Normalizar os dados 

In [12]:
X_train

,area,casal,genero,raca,escolaridade,atividade,aposentadoria,dom_tipo,agua_adeq,esgoto_ad,...,computador,internet,carro,moto,idade,n_moradores,n_criancas,n_adultos,densidade,fone_pc
6882,urbano,casado,mulher,branco,fundam incompleto,ativo ocupado,nao,casa,inadequada,inadequado,...,nao,nao,nao,sim,46,6,0,6,1.200000,1.000000
220,urbano,solteiro,homem,preto ou pardo,medio completo,ativo ocupado,nao,apartamento,adequada,adequado,...,nao,sim,nao,nao,26,2,0,2,0.500000,1.000000
4717,rural,casado,homem,preto ou pardo,fundam incompleto,ativo ocupado,nao,casa,adequada,adequado,...,nao,nao,nao,sim,33,6,4,2,0.857143,0.500000
3694,rural,casado,homem,preto ou pardo,fundam incompleto,ativo ocupado,nao,casa,adequada,inadequado,...,nao,sim,nao,sim,54,4,1,3,0.800000,0.666667
5817,urbano,solteiro,mulher,preto ou pardo,fundam incompleto,inativo,sim,casa,adequada,inadequado,...,nao,sim,nao,nao,70,2,0,2,0.250000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6354,rural,casado,homem,preto ou pardo,fundam incompleto,ativo ocupado,nao,casa,inadequada,adequado,...,sim,sim,sim,sim,33,5,2,4,1.000000,0.500000
3997,urbano,casado,mulher,preto ou pardo,fundam incompleto,ativo ocupado,nao,casa,adequada,inadequado,...,sim,sim,nao,nao,46,2,0,2,0.400000,1.000000
4177,urbano,solteiro,mulher,preto ou pardo,medio completo,ativo ocupado,nao,casa,inadequada,inadequado,...,nao,sim,sim,nao,33,3,2,1,0.750000,3.000000
1055,urbano,casado,mulher,preto ou pardo,medio completo,ativo ocupado,nao,apartamento,adequada,adequado,...,nao,nao,nao,nao,25,2,0,2,1.000000,1.000000


In [13]:
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
X_train[num_vars] = scaler.fit_transform(X_train[num_vars])
X_test[num_vars] = scaler.fit_transform(X_test[num_vars])


### Criar dummies para variáveis categóricas

In [14]:
cat_vars = ['area', 'casal', 'genero', 'raca', 'escolaridade', 'atividade', 'aposentadoria', 
            'dom_tipo', 'agua_adeq', 'esgoto_ad', 'banheiro', 'paredes', 'casa_tipo', 
            'geladeira', 'maquina_lavar', 'tv', 'computador', 'internet', 'carro', 'moto']
X_train = pd.get_dummies(data = X_train, columns = cat_vars)
X_test = pd.get_dummies(data = X_test, columns = cat_vars)

## Selecionando os Atributos

### Definir atributos e variável de resposta no conjunto de treino

In [15]:
X = X_train.values
y = df_train['pobre'].values
feature_names = X_train.columns 

### Teste Anova

In [16]:
selector = feature_selection.SelectKBest(score_func=  
               feature_selection.f_classif, k=15).fit(X,y)
anova_selected_features = feature_names[selector.get_support()]


### Lasso

In [17]:
selector = feature_selection.SelectFromModel(estimator= 
              linear_model.LogisticRegression(C=1, penalty="l1", 
              solver='liblinear'), max_features=15).fit(X,y)
lasso_selected_features = feature_names[selector.get_support()]

### Relevância de atributos 

In [18]:
feature_names = X_train.columns.tolist()

### Importância dos atributos

In [19]:
model = ensemble.RandomForestClassifier(n_estimators=100,
                      criterion="entropy", random_state=0)
model.fit(X,y)
importances = model.feature_importances_

In [20]:
df_importances = pd.DataFrame({"IMPORTANCE":importances, "VARIABLE":feature_names}).sort_values("IMPORTANCE", ascending=False)
df_importances['cumsum'] = df_importances['IMPORTANCE'].cumsum(axis=0)
df_importances = df_importances.set_index("VARIABLE")

## Exportando os dataframes tratados

### Criando dicionario com valores exportados

In [21]:
output = {
    'X_train':X_train,
    'X_test':X_test,
    'y_train':y_train,
    'y_test':y_test,
    'dataframe':df,
    'Anova_features':anova_selected_features,
    'Lasso_features':lasso_selected_features,
    'Random_Forest':model,
    'Importances':df_importances
}

### Exportando os outputs

In [22]:
with open(r"poverty.pickle", "wb") as output_file:
   ...:     cPickle.dump(output, output_file)